In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from numpy.random import choice as choose

import sys
sys.path.insert(0, "../Src/")
import basicUtil

%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
def loadDataWithResizeShape(dataset,resizeShape,split=0):
    if(split == 0):
        print("In non split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51,
                                                                   image_size=resizeShape, batch_size=32)
    else:
        print("In split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51,
                                                                   validation_split=split,subset='both',
                                                                   image_size=resizeShape, batch_size=32)

In [3]:
def createTransferableModel(base_model, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate, L2Rate):
    model = keras.Sequential()
    model.add(base_model)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(L2Rate)))
    model.add(keras.layers.Dropout(dropoutRate))
    model.add(keras.layers.Dense(1,activation='sigmoid'))
            
    adamOptimizer = keras.optimizers.legacy.Adam(learning_rate=adamLearningRate)       
    model.compile(optimizer=adamOptimizer,loss='binary_crossentropy', metrics=basicUtil.f1_score)
    return model

In [4]:
def createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs,
                            dropoutRates,adamLearningRates,L2Rates,trainScores,devScores):
    modelParameters = dict()
    modelParameters['n_NeuronsPerFCLayers'] = n_NeuronsPerFCLayers
    modelParameters['n_Epochs'] = n_Epochs
    modelParameters['dropoutRate'] = dropoutRates
    modelParameters['adamLearningRates'] = adamLearningRates
    modelParameters['L2Rates'] = L2Rates
    modelParameters['trainScore'] = trainScores
    modelParameters['devScore'] = devScores

    modelParametersDF = pd.DataFrame(modelParameters, columns=modelParameters.keys())
    return modelParametersDF

In [5]:
def main():
    imageShape = (180,180)
    inputShape = [imageShape[0],imageShape[1],3]
    train = loadDataWithResizeShape("training",imageShape)
    dev,test = loadDataWithResizeShape("testing",imageShape,.5)
    
    base_model = VGG16(weights="imagenet", include_top=False,input_shape=inputShape,
                      classifier_activation = None,classes = len(train.class_names))
    for layer in base_model.layers:
        layer.trainable = False

    possibleNeuronsPerLayer = basicUtil.createRangeFromMidpoint(16,32)
    possibleEpochs = basicUtil.createRangeFromMidpoint(5,10)

    dropoutCriticalPoints = (0,1)
    adamLearningRateCriticalPoints = (1e-4,1e-2)
    L2CriticalPoints = (1e-2,1e3) 

    n_NeuronsPerFCLayers = []
    n_Epochs = []
    dropoutRates = []
    adamLearningRates = []
    L2Rates = []
    trainScores = []
    devScores = []

    trial = 0
    bestDevScore = 0
    oversizedNeuronNumbers = []
    oversizedEpochNumbers = []
    
    while trial < 100:
        numberOfNeuronsPerFCLayer = choose(possibleNeuronsPerLayer)
        numberOfEpochs = choose(possibleEpochs)
        
        dropoutRate = basicUtil.generateDropoutRate(dropoutCriticalPoints[0],dropoutCriticalPoints[1])
        adamLearningRate = basicUtil.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
        L2Rate = basicUtil.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
        model = createTransferableModel(base_model, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate, L2Rate)
        
        model.fit(train,epochs=numberOfEpochs,verbose=0)

        model_path = f'../Models/VGG16Trials/vgg16_model_{trial}.h5'
        model.save(model_path)
        model_size = os.path.getsize(model_path) / (1024 * 1024)
        if model_size < 60:
            oversizedNeuronNumbers = []
            oversizedEpochNumbers = []
            print('trainScore')
            trainScore = model.evaluate(train)[1]
            print('devScore')
            devScore = model.evaluate(dev)[1]

            if (devScore > 0.91) and (devScore > bestDevScore):
                testScore = model.evaluate(test)[1]
                model_path = f'../Models/best_vgg16_model_.h5'
                model.save(model_path)
                bestModelParams = {
                    'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                    'n_Epochs' : int(numberOfEpochs),
                    'dropoutRate' : dropoutRate,
                    'adamLearningRates' : adamLearningRate,
                    'L2Rates' : L2Rate,
                    'modelSize' : model_size,
                    'trainScore': trainScore,
                    'devScore': devScore,
                    'testScore': testScore
                }
                with open('../Models/best_vgg16_model_params.json', 'w') as f:
                    json.dump(bestModelParams, f)
                bestDevScore = devScore       

            n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
            n_Epochs.append(numberOfEpochs)
            
            adamLearningRates.append(adamLearningRate)
            dropoutRates.append(dropoutRate)
            L2Rates.append(L2Rate)
            trainScores.append(trainScore)
            devScores.append(devScore)
            
            print('concluding trial ',trial)
            trial += 1
        else:
            print(f'redoing trial {trial}. Model was {model_size}MB.')
            failedTrial = createTransferModelParametersDF([numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                                  [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
            display(failedTrial)
            oversizedNeuronNumbers.append(numberOfNeuronsPerFCLayer)
            oversizedEpochNumbers.append(numberOfEpochs)
            if len(oversizedNeuronNumbers) >= 3:
                possibleNeuronsPerLayer = [x for x in possibleNeuronsPerLayer if x < np.mean(oversizedNeuronNumbers)]
                possibleEpochs = [x for x in possibleEpochs if x < np.mean(oversizedEpochNumbers)]
                oversizedNeuronNumbers = []
                oversizedEpochNumbers = []
            
        if (trial % 10 == 9): 
            modelParametersDF = createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs,
                                                    dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
            modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
            display(modelParametersDF)
            
            top5 = modelParametersDF[0:5]
            possibleNumberOfNeuronsPerFCLayer = basicUtil.getAdjustedRange(top5['n_NeuronsPerFCLayers'])
        
            possibleNumberOfEpochs = basicUtil.getAdjustedRange(top5['n_Epochs'])
            dropoutCriticalPoints = basicUtil.calculateCriticalPoints(top5['dropoutRate'])
            adamLearningRateCriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['adamLearningRates'])
            L2CriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['L2Rates'])

            n_NeuronsPerFCLayers = []
            n_Epochs = []
            dropoutRates = []
            adamLearningRates = []
            L2Rates = []
            trainScores = []
            devScores = []

            if bestDevScore > 0.91:
                trial = 101
                
    basicUtil.displayFinalResults('../Models/best_vgg16_model_params.json')
    

In [6]:
main()

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-08-10 23:25:11.016334: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-08-10 23:25:11.016479: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-08-10 23:25:11.254948: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


trainScore
75/75 [==============================] - 94s 1s/step - loss: 0.5565 - f1_score: 0.8966
devScore


2024-08-10 23:31:32.448852: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [299]
	 [[{{node Placeholder/_0}}]]
2024-08-10 23:31:32.449008: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [299]
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 12s 1s/step - loss: 0.5959 - f1_score: 0.8851
concluding trial  0
trainScore
75/75 [==============================] - 96s 1s/step - loss: 0.8579 - f1_score: 0.6711
devScore
10/10 [==============================] - 12s 1s/step - loss: 0.8457 - f1_score: 0.6593
concluding trial  1
trainScore
75/75 [==============================] - 96s 1s/step - loss: 0.9287 - f1_score: 0.9899
devScore
10/10 [==============================] - 12s 1s/step - loss: 0.9285 - f1_score: 0.9901


2024-08-10 23:56:10.018451: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [298]
	 [[{{node Placeholder/_4}}]]
2024-08-10 23:56:10.018573: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [298]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 13s 1s/step - loss: 0.9294 - f1_score: 0.9900
concluding trial  2
trainScore
75/75 [==============================] - 99s 1s/step - loss: 0.9161 - f1_score: 0.6197
devScore
10/10 [==============================] - 12s 1s/step - loss: 0.9206 - f1_score: 0.6380
concluding trial  3
trainScore
75/75 [==============================] - 101s 1s/step - loss: 0.8362 - f1_score: 0.9673
devScore
10/10 [==============================] - 13s 1s/step - loss: 0.8359 - f1_score: 0.9585
concluding trial  4
trainScore
75/75 [==============================] - 102s 1s/step - loss: 2.2675 - f1_score: 0.6606
devScore
10/10 [==============================] - 12s 1s/step - loss: 2.2758 - f1_score: 0.6737
concluding trial  5
trainScore
75/75 [==============================] - 102s 1s/step - loss: 0.7738 - f1_score: 0.9951
devScore
10/10 [==============================] - 12s 1s/step - loss: 0.7740 - f1_score: 0.9951
concluding trial  6
trainScore
75/75 [================

,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
6,1,2,0.901928,0.000517,0.700343,0.995094,0.995094
7,4,6,0.717267,0.000242,0.392362,0.992890,0.992890
2,8,7,0.889545,0.000353,1.003114,0.989896,0.990100
4,4,1,0.749051,0.000101,0.034433,0.967267,0.958533
8,21,1,0.183044,0.001756,0.330105,0.928654,0.918880
0,25,3,0.367919,0.000287,9.093224,0.896641,0.885130
1,16,6,0.998282,0.000169,0.032161,0.671083,0.659280
5,21,8,0.746911,0.006364,34.113856,0.660594,0.673654
3,15,2,0.011316,0.000632,823.514733,0.619699,0.638027


Final Model


,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,modelSize,trainScore,devScore,testScore
0,1,2,0.901928,0.000517,0.700343,56.343903,0.995094,0.995094,0.995094
